In [1]:
# import tensorflow as tf
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras

In [2]:
print(tf.test.is_built_with_cuda())

True


In [4]:
def prepare_tf_data(Train_size = 32, Test_size = 16, img_size = 224, random_ratio = 0.1):
    (train_ds, test_ds), ds_info = tfds.load(
        'cifar10',
        split=['train', 'test'],
        as_supervised=True,
        with_info=True
    )
    
    def preprocess(image, label):
        image = tf.cast(image, tf.float32) / 255.0
        image = tf.image.resize(image, (img_size, img_size))
        return image, label

    def augment(image, label):
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_brightness(image, random_ratio)
        image = tf.image.random_contrast(image, 1 - random_ratio, 1 + random_ratio)
        return image, label
    
    AUTOTUNE = tf.data.AUTOTUNE
    
    train_ds = (train_ds
                .map(preprocess, num_parallel_calls=AUTOTUNE)
                .map(augment, num_parallel_calls=AUTOTUNE)
                .shuffle(10000)
                .batch(Train_size)
                .prefetch(AUTOTUNE))
    
    test_ds = (test_ds
               .map(preprocess, num_parallel_calls=AUTOTUNE)
               .batch(Test_size)
               .prefetch(AUTOTUNE))
    
    return train_ds, test_ds

In [5]:
train_ds, test_ds = prepare_tf_data(Train_size = 64, Test_size = 32,img_size=32)

In [7]:
from models.resnet_cifar import ResNetCIFAR
import tensorflow_addons as tfa

In [10]:
res_ci = ResNetCIFAR(num_classes=10)
# den = tf.keras.models.load_model('./den_cifar10_best')

initial_learning_rate = 1e-2
decay_steps = 5000
# lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
#     initial_learning_rate, decay_steps
# )

lr_schedule_res_ci = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=0.75, 
    staircase=True   
)

res_ci.compile(
    optimizer = tfa.optimizers.AdamW(
    learning_rate=lr_schedule_res_ci,
    weight_decay=0.0001
),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

callbacks_res_ci = [
    tf.keras.callbacks.ModelCheckpoint(
        'res_ci_cifar10_best',  
        save_best_only=True,
        monitor='val_accuracy'
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True
    ),
    tf.keras.callbacks.TensorBoard(log_dir='./logs/res_ci_cifar10')
]

# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")
# model.fit(..., callbacks=[tensorboard_callback])

history_res_ci = res_ci.fit(
    train_ds,
    validation_data=test_ds,
    epochs=20,
    callbacks=callbacks_res_ci
)

Epoch 1/20
782/782 [==============================] - ETA: 0s - loss: 1.7537 - accuracy: 0.3645

INFO:tensorflow:Assets written to: res_ci_cifar10_best\assets


INFO:tensorflow:Assets written to: res_ci_cifar10_best\assets


782/782 [==============================] - 901s 1s/step - loss: 1.7537 - accuracy: 0.3645 - val_loss: 1.7445 - val_accuracy: 0.3914
Epoch 2/20
782/782 [==============================] - ETA: 0s - loss: 1.1153 - accuracy: 0.5987

INFO:tensorflow:Assets written to: res_ci_cifar10_best\assets


...

INFO:tensorflow:Assets written to: res_ci_cifar10_best\assets


782/782 [==============================] - 2313s 3s/step - loss: 0.1294 - accuracy: 0.9556 - val_loss: 0.4315 - val_accuracy: 0.8650
Epoch 19/20
782/782 [==============================] - 2351s 3s/step - loss: 0.1316 - accuracy: 0.9535 - val_loss: 0.4911 - val_accuracy: 0.8568
Epoch 20/20
782/782 [==============================] - ETA: 0s - loss: 0.0983 - accuracy: 0.9661

INFO:tensorflow:Assets written to: res_ci_cifar10_best\assets


INFO:tensorflow:Assets written to: res_ci_cifar10_best\assets


782/782 [==============================] - 2480s 3s/step - loss: 0.0983 - accuracy: 0.9661 - val_loss: 0.4821 - val_accuracy: 0.8671


In [11]:
test_loss, test_accuracy = res_ci.evaluate(test_ds)
print(f"Test accuracy: {test_accuracy:.4f}")

313/313 [==============================] - 69s 220ms/step - loss: 0.4821 - accuracy: 0.8671
Test accuracy: 0.8671
